# <CENTER>  Read Data

In [1]:
import os
import json
import gzip
import pandas as pd
import numpy as np

In [2]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

# <CENTER> Read file

In [3]:
df_M_VG = getDF(r"E:\NIIT\Project\DATA_2014\VG\meta_Video_Games.json.gz")

In [4]:
df_5_VG = getDF(r"E:\NIIT\Project\DATA_2014\VG\reviews_Video_Games_5.json.gz")

# <CENTER> Data preprocessing

In [5]:
df_M_VG.isnull().sum()

asin               0
description     3890
price           7048
imUrl            220
related         5864
salesRank       1937
categories         0
title          48688
brand          50186
dtype: int64

In [6]:
df_M_VG.shape

(50953, 9)

## Exporting various variables to understand

In [7]:
file_name = "description_m_VG14.txt"
descriptions = df_M_VG['description']
with open(file_name, 'w', encoding='utf-8') as f:
    for description in descriptions:
        f.write(str(description) + '\n')

In [43]:
file_name = "salesRank_m_TG14.txt"
salesRank = df_M_VG['salesRank']
with open(file_name, 'w', encoding='utf-8') as f:
    for salesRank in salesRank:
        f.write(str(salesRank) + '\n')

In [44]:
file_name = "title.txt"
title = df_M_VG['title']
with open(file_name, 'w', encoding='utf-8') as f:
    for title in title:
        f.write(str(title) + '\n')

## Renaming the 'asin' and deoping the unnecessary data 

In [7]:
df_temp_MVG=df_M_VG.copy()

In [8]:
df_temp_MVG = df_temp_MVG.rename(columns={'asin': 'ProductID'})

In [9]:
df_temp_MVG=df_temp_MVG.drop(['imUrl','brand','salesRank','description','title'],axis=1)

## Remove Characters "{},(),[]"

In [10]:
import re
def removechar(text):
    special_chars_pattern = r'[\{\}\(\)\[\]]'
    cleaned_text = re.sub(special_chars_pattern, '', text)
    return cleaned_text

In [11]:
C_list=['categories','related']
df_temp_MVG[C_list] = df_temp_MVG[C_list].astype(str)
for col in C_list:
    df_temp_MVG[col] = df_temp_MVG[col].apply(removechar)

In [12]:
df_temp_MVG.head()

,ProductID,price,related,categories
0,0078764343,37.98,"'also_bought': 'B000TI836G', 'B003Q53VZC', 'B0...","'Video Games', 'Xbox 360', 'Games'"
1,043933702X,23.50,"'also_viewed': 'B000067NP1', '0439573947', 'B0...","'Video Games', 'PC', 'Games'"
2,0439339987,8.95,"'also_bought': 'B000314VVU', 'B000PXUOTE', 'B0...","'Video Games', 'PC', 'Games'"
3,0439342260,NaN,'also_viewed': '043934302X',"'Video Games', 'PC', 'Games'"
4,0439339960,NaN,'also_viewed': 'B00028D7TG',"'Video Games', 'PC', 'Games'"


In [25]:
file_name = "categories_mvg14.txt"
categories = df_temp_MVG['categories']
with open(file_name, 'w', encoding='utf-8') as f:
    for categories in categories:
        f.write(str(categories) + '\n')

## Creating sub-Category from category

In [13]:
Temp_var_mvg=df_temp_MVG["categories"].str.split(",", n = 1, expand = True)

In [14]:
df_temp_MVG['Category']=Temp_var_mvg[0]

In [15]:
df_temp_MVG['temp_cat']=Temp_var_mvg[1]

In [16]:
Temp_var_mtg2=df_temp_MVG["temp_cat"].str.split(",", n = 1, expand = True)

In [17]:
df_temp_MVG['Sub-category']=Temp_var_mtg2[0]
df_temp_MVG=df_temp_MVG.drop(['temp_cat','categories'],axis=1)

In [18]:
df_temp_MVG.head(5)

,ProductID,price,related,Category,Sub-category
0,0078764343,37.98,"'also_bought': 'B000TI836G', 'B003Q53VZC', 'B0...",'Video Games','Xbox 360'
1,043933702X,23.50,"'also_viewed': 'B000067NP1', '0439573947', 'B0...",'Video Games','PC'
2,0439339987,8.95,"'also_bought': 'B000314VVU', 'B000PXUOTE', 'B0...",'Video Games','PC'
3,0439342260,NaN,'also_viewed': '043934302X','Video Games','PC'
4,0439339960,NaN,'also_viewed': 'B00028D7TG','Video Games','PC'


In [19]:
df_temp_MVG['Sub-category'].fillna("'Video Games'",inplace=True)

In [20]:
df_temp_MVG.isnull().sum()

ProductID          0
price           7048
related            0
Category           0
Sub-category       0
dtype: int64

In [21]:
Temp_var_mtg3=df_temp_MVG["related"].str.split(":", n = 1, expand = True)

In [22]:
df_temp_MVG['also_buy']=Temp_var_mtg3[0]
df_temp_MVG['other']=Temp_var_mtg3[1]

## Converting "also_buy" column to binary as "yes/no"

In [23]:
for i in range(len(df_temp_MVG['also_buy'])):
    if df_temp_MVG['also_buy'][i] == "'also_bought'":
        df_temp_MVG['also_buy'][i] = '1'
    else:
        df_temp_MVG['also_buy'][i] = '0'

C:\Users\Rishabh\AppData\Local\Temp\ipykernel_23228\2652212497.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp_MVG['also_buy'][i] = '1'
C:\Users\Rishabh\AppData\Local\Temp\ipykernel_23228\2652212497.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp_MVG['also_buy'][i] = '0'


In [24]:
df_temp_MVG=df_temp_MVG.drop(['related','other'],axis=1)

In [25]:
def remove_quotes(text):
    return text.replace("'", '')

df_temp_MVG['Sub-category'] = df_temp_MVG['Sub-category'].apply(remove_quotes)
df_temp_MVG['Category'] = df_temp_MVG['Category'].apply(remove_quotes)

In [26]:
df_temp_MVG['also_buy']=df_temp_MVG['also_buy'].astype(int)

In [27]:
df_temp_MVG.dtypes

ProductID        object
price           float64
Category         object
Sub-category     object
also_buy          int32
dtype: object

In [28]:
def remove_quotes(text):
    return text.replace('"', '')

df_temp_MVG['Sub-category'] = df_temp_MVG['Sub-category'].apply(remove_quotes)
df_temp_MVG['Category'] = df_temp_MVG['Category'].apply(remove_quotes)

In [29]:
df_sort_MVG=df_temp_MVG.copy()

In [30]:
df_sort_MVG.head()

,ProductID,price,Category,Sub-category,also_buy
0,0078764343,37.98,Video Games,Xbox 360,1
1,043933702X,23.50,Video Games,PC,0
2,0439339987,8.95,Video Games,PC,1
3,0439342260,NaN,Video Games,PC,0
4,0439339960,NaN,Video Games,PC,0


In [31]:
len(df_sort_MVG['Sub-category'].unique())

82

In [32]:
from sklearn.preprocessing import LabelEncoder

# Assuming df_sort_MVG is your DataFrame
sub_categories = df_sort_MVG['Sub-category'].unique()

# Initialize the LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(sub_categories)

LabelEncoder()

In [33]:
df_sort_MVG['Sub-category_encoded'] = label_encoder.transform(df_sort_MVG['Sub-category'])

In [34]:
df_sort_MVG.sample(10)

,ProductID,price,Category,Sub-category,also_buy,Sub-category_encoded
363,B00000JX33,NaN,Video Games,Kids & Family,0,37
2252,B00002STY5,NaN,Video Games,More Systems,1,45
5721,B00005B2CW,82.99,Video Games,More Systems,1,45
849,B00001WRJ6,21.99,Video Games,More Systems,0,45
48008,B00E8UH48O,9.99,Video Games,Digital Games,0,22
19160,B000JGD520,49.99,Video Games,Kids & Family,0,37
7690,B000067QD2,NaN,Video Games,More Systems,0,45
19896,B000MRNUB2,3.06,Video Games,Xbox 360,1,78
18516,B000HCJ6QU,59.99,Video Games,PlayStation 3,1,58
5361,B00005444V,NaN,Video Games,More Systems,0,45


In [35]:
df_sort_MVG.set_index('Sub-category_encoded', inplace=True)

## Filling price variable values using interpolate function

In [36]:
df_sort_MVG['price'] = df_sort_MVG['price'].interpolate(method='index', limit_direction="both")

In [37]:
df_sort_MVG.head(10)

,ProductID,price,Category,Sub-category,also_buy
Sub-category_encoded,,,,,
78,0078764343,37.98,Video Games,Xbox 360,1
54,043933702X,23.50,Video Games,PC,0
54,0439339987,8.95,Video Games,PC,1
54,0439342260,19.99,Video Games,PC,0
54,0439339960,19.99,Video Games,PC,0
54,0439374391,20.00,Video Games,PC,0
43,0439394422,12.96,Video Games,Mac,1
43,043940133X,30.00,Video Games,Mac,0
54,0439573947,10.79,Video Games,PC,1


In [38]:
df_sort_MVG.reset_index(inplace=True)
df_sort_MVG.drop('Sub-category_encoded',axis=1,inplace=True)

In [39]:
df_sort_MVG.to_csv('VG_EDA_Data.csv', index=False)

# <CENTER>  5-core Data

In [40]:
df_5_VG.head(2)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2HD75EMZR8QLN,0700099867,123,"[8, 12]",Installing the game was a struggle (because of...,1.0,Pay to unlock content? I don't think so.,1341792000,"07 9, 2012"
1,A3UR8NLLY1ZHCX,0700099867,"Alejandro Henao ""Electronic Junky""","[0, 0]",If you like rally cars get this game you will ...,4.0,Good rally game,1372550400,"06 30, 2013"


In [41]:
df_5_VG.shape

(231780, 9)

In [42]:
df_temp_5VG=df_5_VG.copy()

In [43]:
df_temp_5VG = df_temp_5VG.rename(columns={'asin': 'ProductID'})

In [44]:
df_temp_5VG.isnull().sum()

reviewerID           0
ProductID            0
reviewerName      2813
helpful              0
reviewText           0
overall              0
summary              0
unixReviewTime       0
reviewTime           0
dtype: int64

In [45]:
df_temp_5VG['reviewerName'].fillna('Not_available',inplace=True)

In [46]:
df_temp_5VG.head()

,reviewerID,ProductID,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2HD75EMZR8QLN,0700099867,123,"[8, 12]",Installing the game was a struggle (because of...,1.0,Pay to unlock content? I don't think so.,1341792000,"07 9, 2012"
1,A3UR8NLLY1ZHCX,0700099867,"Alejandro Henao ""Electronic Junky""","[0, 0]",If you like rally cars get this game you will ...,4.0,Good rally game,1372550400,"06 30, 2013"
2,A1INA0F5CWW3J4,0700099867,"Amazon Shopper ""Mr.Repsol""","[0, 0]",1st shipment received a book instead of the ga...,1.0,Wrong key,1403913600,"06 28, 2014"
3,A1DLMTOTHQ4AST,0700099867,ampgreen,"[7, 10]","I got this version instead of the PS3 version,...",3.0,"awesome game, if it did not crash frequently !!",1315958400,"09 14, 2011"
4,A361M14PU2GUEG,0700099867,"Angry Ryan ""Ryan A. Forrest""","[2, 2]",I had Dirt 2 on Xbox 360 and it was an okay ga...,4.0,DIRT 3,1308009600,"06 14, 2011"


## Converting reviewtime to Datetime 

In [47]:
from datetime import datetime

date_format = "%m %d, %Y"
df_temp_5VG['reviewTime'] = pd.to_datetime(df_temp_5VG['reviewTime'], format=date_format).dt.date

In [48]:
df_temp_5VG = df_temp_5VG.rename(columns={'reviewTime': 'Date'})

In [49]:
df_temp_5VG.head(2)

,reviewerID,ProductID,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,Date
0,A2HD75EMZR8QLN,0700099867,123,"[8, 12]",Installing the game was a struggle (because of...,1.0,Pay to unlock content? I don't think so.,1341792000,2012-07-09
1,A3UR8NLLY1ZHCX,0700099867,"Alejandro Henao ""Electronic Junky""","[0, 0]",If you like rally cars get this game you will ...,4.0,Good rally game,1372550400,2013-06-30


In [50]:
df_temp_5VG.drop('unixReviewTime',axis=1,inplace=True)

In [51]:
df_temp_5VG['text']=df_temp_5VG['reviewText']+df_temp_5VG['summary']

In [52]:
df_temp_5VG.drop(['reviewText','summary','helpful'],axis=1,inplace =True)

In [53]:
df_temp_5VG.head()

,reviewerID,ProductID,reviewerName,overall,Date,text
0,A2HD75EMZR8QLN,0700099867,123,1.0,2012-07-09,Installing the game was a struggle (because of...
1,A3UR8NLLY1ZHCX,0700099867,"Alejandro Henao ""Electronic Junky""",4.0,2013-06-30,If you like rally cars get this game you will ...
2,A1INA0F5CWW3J4,0700099867,"Amazon Shopper ""Mr.Repsol""",1.0,2014-06-28,1st shipment received a book instead of the ga...
3,A1DLMTOTHQ4AST,0700099867,ampgreen,3.0,2011-09-14,"I got this version instead of the PS3 version,..."
4,A361M14PU2GUEG,0700099867,"Angry Ryan ""Ryan A. Forrest""",4.0,2011-06-14,I had Dirt 2 on Xbox 360 and it was an okay ga...


## Text preprocessing using SPACY

In [61]:
import spacy
import re

def remove_special_characters(text):
    # Remove all non-letters and non-spaces except for hyphens and digits
    text = re.sub("[^0-9A-Za-z\- ]+", " ", text)
    # Remove unnecessary symbols, punctuation, and signs
    text = re.sub(r'[^\w\s]', '', text)
    # Remove all numbers except those attached to a word
    text = re.sub("(?<!\w)\d+", "", text)
    # Remove all hyphens except between two words
    text = re.sub("-(?!\w)|(?<!\w)-", "", text)
    # Remove multiple spaces
    text = " ".join(text.split())
    return text

In [62]:
nlp = spacy.load("en_core_web_sm")
# def preprocess(text):
#     if text is not np.nan:
#         # Lowercase everything
#         text = text.lower()
#         # Lemmatize the text
#         doc = nlp(text)
#         filtered_tokens = []
#         for token in doc:
#             if token.is_stop:
#                 continue
#             filtered_tokens.append(token.lemma_)
#         # Remove extra spaces
#         return " ".join(filtered_tokens).strip()

## Text preprocessing using NLTK

In [69]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Load the NLTK lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess(text):
    if text is not np.nan:
        # Lowercase everything
        text = text.lower()
        # Tokenize the text
        tokens = nltk.word_tokenize(text)
        filtered_tokens = []
        for token in tokens:
            if token in stop_words:
                continue
            # Lemmatize the token
            lemmatized_token = lemmatizer.lemmatize(token)
            filtered_tokens.append(lemmatized_token)
        # Remove extra spaces
        return " ".join(filtered_tokens).strip()

In [64]:
df_temp_5VG['text']=df_temp_5VG['text'].apply(remove_special_characters)

In [70]:
df_temp_5VG['text']=df_temp_5VG['text'].apply(preprocess)

In [72]:
df_temp_5VG.head()

,reviewerID,ProductID,reviewerName,overall,Date,text
0,A2HD75EMZR8QLN,0700099867,123,1.0,2012-07-09,installing game struggle game window live bug ...
1,A3UR8NLLY1ZHCX,0700099867,"Alejandro Henao ""Electronic Junky""",4.0,2013-06-30,like rally car get game fun oriented european ...
2,A1INA0F5CWW3J4,0700099867,"Amazon Shopper ""Mr.Repsol""",1.0,2014-06-28,st shipment received book instead game nd ship...
3,A1DLMTOTHQ4AST,0700099867,ampgreen,3.0,2011-09-14,got version instead ps3 version turned mistake...
4,A361M14PU2GUEG,0700099867,"Angry Ryan ""Ryan A. Forrest""",4.0,2011-06-14,dirt xbox okay game started playing game lapto...


In [74]:
df_temp_5VG.to_csv('VG_5TG_review.csv', index=False)

## Merging both data 

In [165]:
VG_df = pd.merge(df_sort_MVG, df_temp_5VG, on='ProductID', how='inner')

In [169]:
VG_df.shape

(231780, 10)

In [159]:
VG_df.isnull().sum()

ProductID       0
price           0
Category        0
Sub-category    0
also_buy        0
reviewerID      0
reviewerName    0
overall         0
Date            0
text            0
dtype: int64

# Uploading File as CSV format

In [160]:
VG_df.to_csv('VG_Data.csv', index=False)

## --------------------------------------------------